In [ ]:
import sys
import numpy as np
import requests
import shutil
import mediapipe as mp
import cv2

# Load data

In [ ]:
def load_dataset(data_path):
    # load file
    # convert it into our dataset
    pass

In [ ]:
data_path = "pose_data"

In [ ]:
data_key = sys.argv[1]
# DB = db_pck.load_db(env_var_db_path)
# data_path = load_path(DB.fetch_path(data_key))
dataset = load_dataset(data_path)

# Fisheye correction

In [ ]:
def LoadCameraMatrix(data_path):
    pass

In [ ]:
def CorrectVideo(video, camera_matrix):
    cv.CorrectVideo(video, camera_matrix)

In [ ]:
video =         #dataset.GetVideo()
camera_matrix = LoadCameraMatrix(data_path + "/camera_matrix") # dataset.meta.camera_matrix
c_video = CorrectVideo(video, camera_matrix)

# Pose extraction

In [ ]:
def GetPose(c_video):
    pass

In [ ]:
pose_info = GetPose(video) #GetPose(c_video)

# Unit vector extraction

In [ ]:
def ExtractPoints(pose_info):
    pass

In [ ]:
def PointsToUnitVectors(points, camera_frame):
    pass

In [ ]:
def LoadCameraFrame(data_path):
    pass

In [ ]:
points2D = ExtractPoints(pose_info)
camera_frame = LoadCameraFrame(data_path + "/camera_matrix") #dataset.meta.camera_frame
unit_vectors = PointsToUnitVectors(points2D, dataset.meta.camera_frame)

# Conversion to 3D

In [ ]:
def LeastSquares(unit_vectors, point_distance_pairs):
    pass

In [ ]:
def GetConstPairs(data_path):
    pass

In [ ]:
const_pairs = GetConstPairs(data_path + "/rider_keypointpairs") #dataset.meta.rider_info.GetConstPairs()
points3D = LeastSquares(unit_vectors, const_pairs)

# Points to Area

In [ ]:
def Fit3DMesh(points, mesh):
    pass

In [ ]:
def GetRiderMesh(data_path):
    pass

In [ ]:
mesh = GetRiderMesh(data_path + "/camera_matrix")#dataset.meta.rider_info.GetMesh()
polyline = Fit3DMesh(points3D, mesh)

# Projection to Plane

In [ ]:
def CreatePlane(normal_vec):
    pass

In [ ]:
def ProjectToPlane(polyline, plane):
    pass

In [ ]:
plane = CreatePlane(dataset.wind_direction)
polyshape = ProjectToPlane(polyline, plane)

# Calculate CdA

In [ ]:
def GetArea(polyshape):
    pass

In [ ]:
def CalculateCdA(area):
    pass

In [ ]:
def CalculateDrag(velocity, cda):
    pass

In [ ]:
def CalculateRelative(power, drag):
    pass

In [ ]:
area = GetArea(polyshape)
CdA = CalculateCdA(area)
drag = CalculateDrag(dataset.velocity, CdA)
rel_power = CalculateRelative(dataset.power, drag)

# Visualize

In [ ]:
#play_video(c_video)
#overlay(pose_info)
#overlay(polyshape)

#draw3D(points3D)
#draw3D(polyline)
#draw2D(polyshape)

#drawLabel(area, CdA, drag, rel_power, dataset.velocity, dataset.power)
#drawGraphs(...)